# Assignment 3 Comp 4304
Tyler Strang

# Introduction

This notebook will provide a visualization analysis of the value of various Airbnb rentals, taking into account price, location, and quality of the rental.

# Sources

Main data set was retrieved from a publicly available collection of Airbnb data on New York City rentals that is posted on kaggle (https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data). 

Extra data used in map visual acquired from class notes (ny_geojson.ipnyb)

Graph inspirations/codebase:

Tooltips in some of the graphs were made with inspiration from http://bl.ocks.org/d3noob/a22c42db65eb00d4e369

Interactive bar chart code was started with example from book by Scott Murray - Interactive Data Visualization for the web 2nd edition. Inspiration for it was the class on transitions (note 16)

Stack Bar chart was initially made with reference to https://bl.ocks.org/mjfoster83/7c9bdfd714ab2f2e39dd5c09057a55a0, later updated to incorporate features from class examples (specifically statestack4.html). My stack chart differs from the class example in that the dataset differs and had to be compiled into the proper form, a mouseover tooltip is incorporated, the "line up stacks" feature has a different implementation, as well as other small visual variations.

Box plot was started using this example - https://www.d3-graph-gallery.com/graph/boxplot_basic.html . My bar chart differs in that it has several plots on one graph, it was made from a dataset that had to be compiled into the proper form, and it incorporates a mouseover tooltip, as well as other small visual variations.

Interactive Map graph was made with refernce to class notes, specifically note 17-19 and ny_geojson.ipnyb. My chart incorporates many of the features across these notes. Heatmap on map inspiration was this kaggle notebook for the main data set - https://www.kaggle.com/dgomonov/data-exploration-on-nyc-airbnb

# Data Definition

In [1]:
import pandas as pd
airbnbData = pd.read_csv('AB_NYC_2019.csv')
print("Airbnb Data Columns: ")
for x in airbnbData.columns:
    print(x)

Airbnb Data Columns: 
id
name
host_id
host_name
neighbourhood_group
neighbourhood
latitude
longitude
room_type
price
minimum_nights
number_of_reviews
last_review
reviews_per_month
calculated_host_listings_count
availability_365


__AirBnb Data Columns:__

id: Unique id of the rental listing (integer number)

name: Name of the rental listing (string of words, most often a very short description of the place)

host_id: Unique id of the host of this rental listing (integer number)

host_name: Name of the host of this rental listing (string, usually the hosts first name, but may be several hosts)

neighborhood_group: The borough of New York the rental is in (string, values can be Manhattan, Bronx, Queens, Brooklyn, or Staten Island)
 
neigborhood: The neighborhood the rental is in (string)

latitude: The latitude coordinates of the location of the listing 

longitude: The longitude coordinates of the location of the listing

room_type: The type of rental unit (string, options are shared room, private room,or entire home/apt)

price: Price per night of the rental unit (integer price in dollars)

minimum_nights: The minimum number of nights you must rent this unit for (integer number)

number_of_reviews: The number of reviews this rental listing has (integer number) 

last_review: The date the last review of the listing was posted (YYYY-MM-DD format)

reviews_per_month:The average number of reviews per month (floating point number)

calculated_host_listings_count:The total number of rental units the host of this property manages (integer number)

availability_365: The number of days this rental unit is available for (integer number)

# Munzner Analysis and Visualization of Problem

The problem presented is:
>At some time in the future, I want to visit some interesting sites on a trip to New York City, and I will probably use an airbnb host. I wonder which neighbourhood offers the best value for accomodations. I have a limited budget, so I might choose to be futher from the interesting sites if I could save some money. I would also pay a bit more for better accomodations.

And as part of solving this problem, we must make 4 plots using d3 that relate to this request.

**What** analysis:

Our given dataset to solve this problem is a static, flat table. In Munzner's framework, a table is a dataset where the data is stored in rows and columns, where the rows are various items and each column corresponds to an attribute. The table contains spatial data as well(lat and lon) that corresponds to the geometry of New York.  The data that we are concerned with is closely linked to _ separate attributes in AB_NYC_2019.csv:

neighborhood_group: A categorical attribute denoting the borough that the rental unit is located in. The problem stated is concerned with which neigborhood has the best value, and is interested in the rental units proximity to sites of interest, so this attribute is of interest.

latitude and longitude: Two quantitative, spatial field attributes that give the exact location of the rental unit. These attributes are important if we end up visulizing the data in terms of the geometry of New York.

price: A quantitative attribute listing the price of the rental per night. Problem is concerned with value, so price will be a factor no matter what else is taken into account.

room_type: A categorical attribute denoting whether the rental listing is an entire home/apt, a single room, or a shared room. Problem mentions possibly staying somewhere further from sights for a higher quality stay, so the room type may be important for the value of the user.


The problem makes note twice of the fact that the user is going to see interesting sites. Since the given dataset does not include any such data, we will also be using a dataset from NYC open data to plot places of interest. The dataset used (NYC_POImodified.geojson) compiles points of interest in NYC as catalogged by various city agencies, and this dataset has been filtered to only include those place of interest that are categorized as a Cultural facility or Recreational facility. This data file, as a json structure, is a tree, where each item in the datafile has child data structures that refer to the parent node. Given we only care about a few points of data, we will mostly be accessing this data in the same way we would a static flat table. The data from this set we are most concerned with is:

properties.name: A categorical attribute denoting the name of the associated facility. The information of the facility gathered from its name is what will be presented to the user so that they may gauge if it is of interest to them.

d.geometry.coordinates: Two quantitative, spatial field attributes (lon and lat) that give the exact location of the facility. These attributes are important if we end up visulizing the data in terms of the geometry of New York.

**WHY** analysis:

Munzner's **why** analysis is split into two parts: The action and the target. The action component is further split into three levels: analyze, search and query. This problem is broad and multifaceted, asking to take into account different factors like the value of accomadations in specific neigborhoods, proximity to sites of interest, and paying more for better quality of accomadations. We should keep these things in mind when determining the actions and targets.

Analyze: The highest level of user goal is how the user wants to analyze the data. The one that seems to fit the most for the problem is **discover**. The user giving the problem statement is trying to find suitable accomadations for a future trip,and so we want to show them a visual that will allow them to find a rental unit/neigbourhood that best fits their criteria. We want to show them how the rentals fit against their variables of interest, but allow them to find their best fit on their own. We are not showing them only the rentals that we have determined best fit their criteria (that would be present), but showing them a range of rentals that may fit their criteria, and let them discover on their own which is the best fit.

Search: All visualizations require the user to search for targets of interest. Determining which of the 4 search classifications our problem falls into depends on whether the target we are looking for is known or unknown, and whteher the location of our target is known or unknown. This problem makes it hard to pin down exactly what the client is looking for. They want to find the best value unit/neigborhood (itself vague, value is price versus what?), as well as proximity to sites of interest, and the quality of accomadations. Therefore, it seems likely that the user doesn't have a specific target in mind, rather is hoping to look at many available options and choose one that suits their needs. The location is known however. While the user does not know exactly what they're looking for, they will know to look at specific places (where prices are low,where close to where they want to visit, etc.). Therefore the search goal that fits best for our problem is **browse**.

Query: The query goal that seems to fit the problem best is **comparision**. The user wants to determine the best value for their own budget and interests, and since we know some of their preferences (low price, good quality) it's best to provide the user the rental units/neigborhoods that best adhere to their preferences, and allow them to select which best suits them. Summarize might seem like it could work for this at a glance, but the user is not interested in every single rental (those with an exceedingly high price point for example), and so showing them everything is more than they need to know. Comparison between relevant rentals is more useful than a high level summary.

The second part of the why analysis is the target. Since the user's end goal is to find rentals/neigbourhoods with the best value, the targets that make the most sense are **outliers**, rental units and neigborhoods that have an exceptionally low price/good quality, or **features**, a target that is task dependent, but in this case we can take to mean the value of the units/neigborhoods in terms of the balance between price, location, and quality.

Boiling the task down to a two sentence action target pair, I believe the most useful way to tackle the problem is by creating visualizations that allow the user to **browse features**. This seems to be most in the spirit of the problem, as the user has the vague task of finding the units/neigborhoods that best fit their definition of value, and allowing them to browse the how the various aspects of their value relation connect together is the bect way to do that.

# Graphs and How Analysis

# Interactive map

Map of New York that can be zoomed and panned around. Mousing over the yellow circles (places of interest) will show
a tooltip with the name of the attraction, while mousingover a white-green circle will show the rental price per night of the place as well as the room type and neigborhood. The attached checkboxes can be used to turn off the poi circles, the rental unit circles, and the streets. Hitting the reset button will reset the pan and zoom to the original place.

In [2]:
%%javascript

require.config({
    paths: {
        d3v4: 'https://d3js.org/d3.v4.min'
    }
});

<IPython.core.display.Javascript object>

In [3]:
%%HTML
<style>div.tooltip {
    position: absolute;
    text-align: center;
    width: 80px;
    height: 70px;
    padding: 2px;
    font: 10px sans-serif;
    background: lightsteelblue;
    border: 0px;
    border-radius: 8px;
    pointer-events: none;
}
</style>

<svg id="NYCmap"></svg>
<svg height="100" width="230">
  <defs>
    <linearGradient id="grad1" x1="0%" y1="0%" x2="0%" y2="100%">
      <stop offset="0%" style="stop-color:white;stop-opacity:1" />
      <stop offset="100%" style="stop-color:darkgreen;stop-opacity:1" />
    </linearGradient>
  </defs>
<text y = "10" x = "15"> Price-0</text>
<text y = "90" x = "15"> Price-400</text>
  <rect x="10" y="0" width="5" height="95" fill="url(#grad1)" />
    <circle cx = "95" cy = "10" r = "5" fill = "#67a367" stroke = "black" />
    <text y = "15" x = "115">- Rental Unit</text>
    <circle cx = "95" cy = "40" r = "5" fill = "yellow" />
    <text y = "45" x = "115">- Place of Interest</text>
</svg> 
<ul><li> <input type="checkbox" id="rentalCheckbox" checked="true" >Rental Units</input></li>
<li><input type="checkbox" id="poicCheckbox" checked="true" >Places of Interest</input></li>
<li><input type="checkbox" id="streetCheckbox" checked="true" >Streets</input></li>
<li><button id="resetMap">Reset</button></li></ul>

In [4]:
%%javascript
require(['d3v4'], function(d3) {
    //Width and height
var w = 700;
var h = 400;

function cvert(d) {
    if (parseInt(d.last_review) == 2019) {
        if (parseInt(d.price) < 400) {
            if (parseInt(d.id) % 13 == 0) {
                return {
                    latitude: d.latitude,
                    longitude: d.longitude,
                    neighborhood : d.neighbourhood,
                    room_type : d.room_type,
                    price: parseFloat(d.price)
                }
            }
        }
    }
}

var colorScale = d3.scaleLinear()
    .domain([0.0, 400.0])
    .range(["white", "darkgreen"]);

//Define map projection
var nycProj = d3.geoOrthographic() //NYC
    .rotate([74.006, -40.7128]);

//Define path generator
var nycPathFn = d3.geoPath()
    .projection(nycProj);

var center = nycProj([-74.006, 40.7128]);

//Locate SVG element
var svgSelection = d3.select("#NYCmap")
    .attr("width", w)
    .attr("height", h);


//Define what to do when panning or zooming
var zooming = function(d) {

    //Update projection with new offset and scale
    nycProj.translate([d3.event.transform.x, d3.event.transform.y])
        .scale(d3.event.transform.k * 40000);

    //Update all paths and circles
    svgSelection.selectAll("path")
        .attr("d", nycPathFn);

    svgSelection.selectAll(".rentals")
        .attr("cx", d => nycProj([d.longitude, d.latitude])[0])
        .attr("cy", d => nycProj([d.longitude, d.latitude])[1]);

    svgSelection.selectAll(".poic")
        .attr("cx", d => nycProj(d.geometry.coordinates)[0])
        .attr("cy", d => nycProj(d.geometry.coordinates)[1]);
}

var zoom = d3.zoom()
    .scaleExtent([0.7, 37.0])
    .translateExtent([
        [-700, -700],
        [700, 700]
    ])
    .on("zoom", zooming);
    
var div = d3.select("body").append("div")
    .attr("class", "tooltip")
    .style("opacity", 0);

var map = svgSelection.append("g")
    .attr("id", "map")
    .call(zoom) //Bind the zoom behavior
    .call(zoom.transform, d3.zoomIdentity //Then apply the initial transform
        .translate(w, h)
        .scale(1.0)
        .translate(-center[0], -center[1]));

//Create a new, invisible background rect to catch zoom events
map.append("rect")
    .attr("x", 0)
    .attr("y", 0)
    .attr("width", w)
    .attr("height", h)
    .attr("opacity", 0);
//Load in GeoJSON data
d3.json("Boroughs.geojson", function(boros) {
    //Bind data and create group with text and one path per GeoJSON feature
    var entrySelection = map.selectAll("g")
        .data(boros.features).enter()
        .append("g");
    entrySelection
        .append("path")
        .attr("d", nycPathFn)
        .style("stroke", "black")
        .style("fill", "none")
        .each(d => {
            d.centroid = nycPathFn.centroid(d);
        });
});

d3.csv("AB_NYC_2019.csv", cvert, function(dataset) {
    svgSelection.selectAll(".rentals")
        .data(dataset).enter()
        .append("circle")
        .classed("rentals", true)
        .attr("r", 3)
        .attr("cx", d => nycProj([d.longitude, d.latitude])[0])
        .attr("cy", d => nycProj([d.longitude, d.latitude])[1])
        .attr("stroke", "black")
        .style("fill", d => colorScale(d.price))
        .on("mouseover", function(d) {
            div.transition()
                .duration(200)
                .style("opacity", .9);
            div.html("Price: " + d.price + "$ <br/> Neigborhood: "  + d.neighborhood + "<br/> Room Type: " + d.room_type)
                .style("left", (d3.event.pageX) + "px")
                .style("top", (d3.event.pageY - 28) + "px");
            })
        .on("mouseout", function(d) {
            div.transition()
                .duration(500)
                .style("opacity", 0);
    });
});
d3.json("NYC_POImodified.geojson", function(poi) {
    svgSelection.selectAll(".poic")
        .data(poi.features).enter()
        .append("circle")
        .classed("poic", true)
        .attr("r", 2)
        .attr("cx", d => nycProj(d.geometry.coordinates)[0])
        .attr("cy", d => nycProj(d.geometry.coordinates)[1])
        .attr("fill", "yellow")
    .on("mouseover", function(d) {
            div.transition()
                .duration(200)
                .style("opacity", .9);
            div.html(d.properties.name + "<br/>")
                .style("left", (d3.event.pageX) + "px")
                .style("top", (d3.event.pageY - 28) + "px");
            })
        .on("mouseout", function(d) {
            div.transition()
                .duration(500)
                .style("opacity", 0);
    });
});
d3.json("NYC_CSCL_modified.geojson", function(streets) {
    var entrySelection = svgSelection.selectAll(".streets")
        .data(streets.features).enter()
        .append("path")
        .classed("streets", true)
        .attr("d", nycPathFn)
        .attr("stroke", "black");
});
    
    d3.select("#resetMap")
        .on("click", function() {
          map.transition()
            .call(zoom.transform, d3.zoomIdentity 
            .translate(w, h)
            .scale(1.0)
            .translate(-center[0], -center[1]));
      });
d3.select("#rentalCheckbox")
    .on("click", function() {
        var circlesvis = d3.select("#rentalCheckbox").property("checked") ?
            "visible" : "hidden"
        svgSelection.selectAll(".rentals").attr("visibility", circlesvis)
    });
d3.select("#poicCheckbox")
    .on("click", function() {
        var circlesvis = d3.select("#poicCheckbox").property("checked") ?
            "visible" : "hidden"
        svgSelection.selectAll(".poic").attr("visibility", circlesvis)
    });
d3.select("#streetCheckbox")
    .on("click", function() {
        var circlesvis = d3.select("#streetCheckbox").property("checked") ?
            "visible" : "hidden"
        svgSelection.selectAll(".streets").attr("visibility", circlesvis)
    });
    });

<IPython.core.display.Javascript object>

Effectiveness for end user:

This plot is useful to the end user because it will allow them to browse a large number of features at one time. Panning and zooming the map, as well as checkboxes that allow the user to filter what they want to see at one time are features that enchance the browsing capabilities of the user. Having information about rental price and NYC places of interest laid out on a map of the city allows the user see where each of the variables are in relation to each other, where there are clusters of sparsely populated areas, and other features. User has information available to them to make judgements on their personal best value

How:

Idiom: Doesn't match a predefined idiom in Munzner, has elements of chloropleth map. Geometry data is used to make boundaries for a map to be used as the canvas, and geometry data is used to plot the location of points that use diverging colormap to denote quantitative variable. 

Marks: 

Area - Geometrical map data drawn to scale

Lines - Geometry of streets are lines (laid on map)

Points - Locations on map (POI and rental units) are points laid on the map area

Channels:

Color Saturation - Green to white diverging color channel used to express magnitude (rental price)

Color Hue - Yellow and the aforementioned green-white circles use different hues to differentiate between category (rental unit or POI)

Spatial Region - Map boundaries, streets,POI and Rental Unit spaces denote where they exist on the map. 

Other Encodings:

Manipulate - Navigate - Panning and zooming allow user to browse spatial map

Filter - Mouseover of a point provides data directly about that point

Effectiveness tied to HOW elements:
The Points of rental units and POI laid out on a geographical map area provide an effective way to visualize where these areas are in relation to eachother. The price of units uses a diverging saturation scale which provides an estimate of the price in relation to others at a glance. Rental units are distinct color from POI, which Munzner denotes is the second best method at differentiating categorical attributes. The fact that the map can filter information with the checkboxes, and the data can be navigated, cuts down on information overload by allowing user to see what they need.


# Price Box Plots

This graph is a series of box plots of rental prices, one for each borough. Mousing over the IQR rectangle will show a tooltip with the values for min, q1,median, q3,and max for the moused over borough.

In [5]:
%%HTML
<h1> Box Plots of Prices per Borough </h1>
<div id="my_dataviz"></div>

In [6]:
%%javascript
require(['d3v4'], function(d3) {
var margin = {
        top: 10,
        right: 30,
        bottom: 30,
        left: 40
    },
    width = 900 - margin.left - margin.right,
    height = 400 - margin.top - margin.bottom;

var svgBox = d3.select("#my_dataviz")
    .append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
    .append("g")
    .attr("transform",
        "translate(" + margin.left + "," + margin.top + ")");
    
svgBox.append("text")
        .attr("class", "priceLabel")
        .attr("transform", "rotate(-90)")
        .attr("y", -40)
        .attr("x", 0 - (height / 2))
        .attr("dy", "0.7em")
        .style("text-anchor", "middle")
        .text("Price(in US dollars)");


var yScale = d3.scaleLinear()
    .range([height, 0]);


var xScale = d3.scaleBand()
    .range([0, width]);


var center = 200;
var barWidth = 50;


d3.csv("boxPlotData.csv", function(dataset) {
    var max = d3.max(dataset, function(d) {
        return +d.max;
    });
    yScale.domain([0, max])
    xScale.domain(dataset.map(function(d) {
        return d.borough;
    }));
    svgBox.append("g").call(d3.axisLeft(yScale));
    svgBox.append("g").attr("transform", "translate(0," + height + ")").call(d3.axisBottom(xScale));

    svgBox.selectAll(".line").data(dataset).enter()
        .append("line")
        .attr("class", "line")
        .attr("x1", d => xScale(d.borough) + xScale.bandwidth() / 2)
        .attr("x2", d => xScale(d.borough) + xScale.bandwidth() / 2)
        .attr("y1", d => yScale(+d.min))
        .attr("y2", d => yScale(+d.max))
        .attr("stroke", "black");


    svgBox.selectAll(".boxRect")
        .data(dataset)
        .enter()
        .append("rect")
        .attr("class", "boxRect")
        .attr("x", d => xScale(d.borough) + xScale.bandwidth() / 2 - barWidth / 2)
        .attr("y", d => yScale(+d.q3))
        .attr("height", d => (yScale(+d.q1) - yScale(+d.q3)))
        .attr("width", barWidth)
        .attr("stroke", "black")
        .style("fill", "#69b3a2")
        .on("mouseover", function() {
            tooltipBox.style("display", null);
        })
        .on("mouseout", function() {
            tooltipBox.style("display", "none");
        })
        .on("mousemove", function(d) {
            var xPosition = d3.mouse(this)[0] - 5;
            var yPosition = d3.mouse(this)[1] - 125;
            tooltipBox.attr("transform", "translate(" + xPosition + "," + yPosition + ")")
            tooltipBox.select(".mintext").text("Min: " + d.min)
            tooltipBox.select(".q1text").text("Q1: " + d.q1)
            tooltipBox.select(".medtext").text("Median: " + d.median)
            tooltipBox.select(".q3text").text("Q3: " + d.q1)
            tooltipBox.select(".maxtext").text("Max: " + d.max);
        });

    svgBox.selectAll(".minLine").data(dataset).enter()
        .append("line")
        .attr("class", "minLine")
        .attr("x1", d => xScale(d.borough) + xScale.bandwidth() / 3)
        .attr("x2", d => xScale(d.borough) + xScale.bandwidth() - xScale.bandwidth() / 3)
        .attr("y1", d => yScale(+d.min))
        .attr("y2", d => yScale(+d.min))
        .attr("stroke", "black");

    svgBox.selectAll(".maxLine").data(dataset).enter()
        .append("line")
        .attr("class", "maxLine")
        .attr("x1", d => xScale(d.borough) + xScale.bandwidth() / 3)
        .attr("x2", d => xScale(d.borough) + xScale.bandwidth() - xScale.bandwidth() / 3)
        .attr("y1", d => yScale(+d.max))
        .attr("y2", d => yScale(+d.max))
        .attr("stroke", "black");

    svgBox.selectAll(".medLine").data(dataset).enter()
        .append("line")
        .attr("class", "medLine")
        .attr("x1", d => xScale(d.borough) + xScale.bandwidth() / 2 - barWidth / 2)
        .attr("x2", d => xScale(d.borough) + xScale.bandwidth() / 2 - barWidth / 2 + barWidth)
        .attr("y1", d => yScale(+d.median))
        .attr("y2", d => yScale(+d.median))
        .attr("stroke", "black");
    
    var tooltipBox = svgBox.append("g")
        .attr("class", "tooltipBox")
        .style("display", "none")
        .style("pointer-events", "none");

    tooltipBox.append("rect")
        .attr("width", 80)
        .attr("height", 150)
        .attr("fill", "white")
        .style("opacity", 0.5);

    tooltipBox.append("text")
        .attr("class", "q1text")
        .attr("x", 40)
        .attr("dy", "15%")
        .style("text-anchor", "middle")
        .attr("font-size", "12px")
        .attr("font-weight", "bold");
    tooltipBox.append("text")

        .attr("class", "q3text")
        .attr("x", 40)
        .attr("dy", "25%")
        .style("text-anchor", "middle")
        .attr("font-size", "12px")
        .attr("font-weight", "bold");

    tooltipBox.append("text")
        .attr("class", "medtext")
        .attr("x", 40)
        .attr("dy", "20%")
        .style("text-anchor", "middle")
        .attr("font-size", "12px")
        .attr("font-weight", "bold");

    tooltipBox.append("text")
        .attr("class", "mintext")
        .attr("x", 40)
        .attr("dy", "10%")
        .style("text-anchor", "middle")
        .attr("font-size", "12px")
        .attr("font-weight", "bold");

    tooltipBox.append("text")
        .attr("class", "maxtext")
        .attr("x", 40)
        .attr("dy", "30%")
        .style("text-anchor", "middle")
        .attr("font-size", "12px")
        .attr("font-weight", "bold");

});
});

<IPython.core.display.Javascript object>

Effectiveness for end user:

This plot is useful to the end user because it will allow them to browse the pricing features for each borough.At a glance the user can see the spread of prices at each borough, and the symmetry and skewness of those prices.

How:

Idiom - Box Plot. Five quantitative attributes are derived from dataset of many quantitave attributes denoting their distributions. There is one box plot for each categorical attribute (borough)

Marks:

Lines - 3 of the 5 derived attributes are denoted by lines, as well as the space between points

Channels:

Length - Length of the "box" and length of the "whiskers" denote the difference between the derived attributes min-max and q1-q3

Spatial Region - Used to express categories (boroughs). Highest ranked channel in munzners effectiveness of categorical attributes 

Position on a common scale - Different endpoints of the box and whiskers are all measured on common scale of quantitaive attribute(price). Most effective channel for quantitative attributes according to Munzner

Other Encodings:

Filter - Mouseover shows raw data to user.

Effectiveness tied to HOW elements:
The box plot utilizes two of munzners most highly rated channels (common scale and spatial region) to show the distribution of derived attributes. Lengths of box plots boxes and whiskers shown next to each other effectively display the skewness or symmetry of the data it was derived from.

# Stacked bar chart of room types

Each bar has the count of the total number or rental units for each borough, with the stacked portions showing the count of each room type available in that location. Mousing over any stack-bar will give the exact count of that room type. Hitting the "Stack View" button will seperate each stack level so that they rest on the same base (and move them so they don't hit the other stacks), and hitting it again will revert the view.

In [7]:
%%HTML
<style> tooltipStack {
    pointer-events: none;
}
</style>
<h1> Count of Room Types per Borough </h1>
<button type="button" id = "layerButton">Stack View</button>
<svg class = "stackChart"  width="900" height="500"></svg>

In [8]:
%%javascript
require(['d3v4'], function(d3) {
var stackSplit = false;

var svgStack = d3.select(".stackChart"),
    margin = {
        top: 20,
        right: 20,
        bottom: 30,
        left: 40
    },
    width = +svgStack.attr("width") - margin.left - margin.right,
    height = +svgStack.attr("height") - margin.top - margin.bottom,
    gStack = svgStack.append("g").attr("transform", "translate(" + margin.left + "," + margin.top + ")");


var xScale = d3.scaleBand()
    .rangeRound([0, width])
    .paddingInner(0.05)
    .align(0.1);


var yPos = d3.scaleLinear()
    .rangeRound([height, 0]);
var ySize = d3.scaleLinear()
    .rangeRound([0, height]);

var zScale = d3.scaleOrdinal(d3.schemeCategory10);

var valuekeys = ["Entire home/apt", "Private room", "Shared room"]

function cvert(d) {
    d.total = 0;
    d.cumulative = []
    valuekeys.forEach(function(key) {
        d[key] = parseInt(d[key]);
        d.cumulative.push([d[key], d.total]);
        d.total += d[key];
    });
    return d;
}
d3.csv("convertcsv.csv", cvert, function(data) {


    data.sort(function(a, b) {
        return b.total - a.total;
    });
    xScale.domain(data.map(d => d.room_type));
    yPos.domain([0, d3.max(data, d => d.total)]);
    ySize.domain([0, d3.max(data, d => d.total)]);
    zScale.domain(valuekeys);

    var layers = gStack.selectAll(".boroughBar")
        .data(data)
        .enter()
        .append("g")
        .attr("class", "boroughBar")
        .attr("transform", d => "translate(" + (xScale(d.room_type)) + ", 0)")
        .on("mousemove", function(d) {
            var xPosition = d3.mouse(this)[0] - 5 + xScale(d.room_type);
            var yPosition = d3.mouse(this)[1] - 5;
            tooltip.attr("transform", "translate(" + xPosition + "," + yPosition + ")");
        });

    var boroughRects = layers.selectAll("rect")
        .data(d => d.cumulative)
        .enter()
        .append("rect")
        .attr("y", d => yPos(d[1]) - ySize(d[0]))
        .attr("height", d => ySize(d[0]))
        .attr("width", xScale.bandwidth())
        .attr("fill", (d, i) => zScale(valuekeys[i]))
        .on("mouseover", function() {
            tooltip.style("display", null);
        })
        .on("mouseout", function() {
            tooltip.style("display", "none");
        })
        .on("mousemove", function(d) {
            tooltip.select("text").text(d[0]);
        });

    var splitY = valuekeys.map(function(k, i) {
        return d3.max(data, d => d.cumulative[i][1])
    });

    function splitstacks() {
        boroughRects.transition().duration(2000)
            .attr("width", xScale.bandwidth() / 3)
            .attr("x", (d, i) => xScale.bandwidth() / 3 * i)
            .attr("y", (d, i) => yPos(splitY[i]) - ySize(d[0]));
    }

    function mergestacks() {
        boroughRects.transition().duration(2000)
            .attr("x", 0)
            .attr("width", xScale.bandwidth())
            .attr("y", (d, i) => yPos(d[1]) - ySize(d[0]))
    }

    d3.select("#layerButton")
        .on("click", function() {
            stackSplit = !stackSplit;
            if (stackSplit) {
                splitstacks();
            } else {
                mergestacks();
            }
        });


    gStack.append("g")
        .attr("class", "axis")
        .attr("transform", "translate(0," + height + ")")
        .call(d3.axisBottom(xScale));

    gStack.append("g")
        .attr("class", "axis")
        .call(d3.axisLeft(yPos).ticks(null, "s"))
        .append("text")
        .attr("x", 2)
        .attr("y", yPos(yPos.ticks().pop()) + 0.5)
        .attr("dy", "0.32em")
        .attr("fill", "#000")
        .attr("font-weight", "bold")
        .attr("text-anchor", "start");

    var legend = gStack.append("g")
        .attr("font-family", "sans-serif")
        .attr("font-size", 10)
        .attr("text-anchor", "end")
        .selectAll("g")
        .data(valuekeys.slice().reverse())
        .enter().append("g")
        .attr("transform", function(d, i) {
            return "translate(0," + i * 20 + ")";
        });

    legend.append("rect")
        .attr("x", width - 19)
        .attr("width", 19)
        .attr("height", 19)
        .attr("fill", zScale);

    legend.append("text")
        .attr("x", width - 24)
        .attr("y", 9.5)
        .attr("dy", "0.32em")
        .text(function(d) {
            return d;
        });
});


var tooltip = svgStack.append("g")
    .attr("class", "tooltipStack")
    .style("display", "none");

tooltip.append("rect")
    .attr("width", 60)
    .attr("height", 20)
    .attr("fill", "white")
    .style("opacity", 0.5);

tooltip.append("text")
    .attr("x", 30)
    .attr("dy", "1.2em")
    .style("text-anchor", "middle")
    .attr("font-size", "12px")
    .attr("font-weight", "bold");
});

<IPython.core.display.Javascript object>

Effectiveness for end user:

This plot is useful to the end user because it will allow them to browse the number of various room types available in an easy way. The bars give a good visual approximation to the user of how many of each room type is available, and mousing over the bar will give the exact count, allowing the user to quickly browse the data and find what they are looking for. Mouseover and "stack view" buttons provide more options for browsing and finding features.


How:

Idiom - Stacked Bar Chart. Categorical attributes each have an associated quantitave attribute denoted by length of bar. This bar is divided into sections for a second type of categorical attribute for each bar.

Marks:

Lines - Bars (thick lines) are used to denote quantitative attribute

Area - Bars are seperated into seperate sections by categorical attribute.

Channels:

Length - Length of the bars denote amount of quantitative data per each cateogrical attribute

Spatial Region - Used to express categories (boroughs). Highest ranked channel in munzners effectiveness of categorical attributes

Color Hue - Used to denote the sub-bars within each main bar.

Other Encodings:

Filter - Button click to seperate the subdivided bars and align each of the "inner" categorical attributes. Mouseover provides raw count

Effectiveness tied to HOW elements: Bars use length to denote amount of qualitative attribute for each categorical. Since the graph is following Munzner's Seperate, Ordered, and Aligned rule of thumb, the relative value for each category compared to each other is clear. The ability to filter the room_type categorical attribute to align on a common scale allows the rule of thumb to apply to this inner category and show the relative value between these types more effectively. The color hue cleanly delineates between the three room types.


# Original Design: Interactive Average Price Bar Graph

Shows average rental price per borough/neighborhood. You can click on the bar for a borough, and the graph will change
to show the ten lowest-average priced neighborhoods within that borough. Mousing over the bar will show the exact price.
The "Flip it!!!" button will change the graph from vertical bars to horizontal. The "Reset to Boroughs" button will reset the bars from
neighbourhood bars back to boroughs

In [9]:
%%HTML
<style> g.tooltipBar {
    pointer-events: none;
}
</style>
<h1>Average price per Borough/Neighborhood</h1>
<ul><li><button type="button" id = "resetButton">Reset to Boroughs</button> </li>
<li><button type="button" id = "flip">Flip it!!!!</button> </li></ul>
<svg class="graph"></svg>

In [10]:
%%javascript
require(['d3v4'], function(d3) {
var w = 850;
var h = 300;
var flipped = false;
var padding = 20;
var globalBorough;
var svgEl = d3.select(".graph")
    .attr("width", w)
    .attr("height", h);
var margin = {
    top: 20,
    right: 30,
    bottom: 35,
    left: 100
};
var width = w - margin.left - margin.right;
var height = h - margin.top - margin.bottom;
var gCanvas = svgEl.append("g").attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var formatDecimals = d3.format(".3");

var colorScale = d3.scaleOrdinal(d3.schemeCategory10);

function cvert(d) {
    if (parseInt(d.last_review) == 2019) {

        return {
            borough: d.neighbourhood_group,
            neighbourhood: d.neighbourhood,
            price: parseFloat(d.price)
        }
    }
}

function convertToNeigborhood(d, neigborhood) {
    if (d.borough == neigborhood) {
        return {
            neighbourhood: d.neighbourhood,
            price: parseFloat(d.price)
        }
    }
}

function makeTheGraph() {
    d3.csv("AB_NYC_2019.csv", cvert, function(dataset) {



        var avgByBorough = d3.nest()
            .key(d => d.borough)
            .rollup(function(v) {
                return d3.mean(v, function(d) {
                    return d.price;
                });
            })
            .entries(dataset).sort(function(a, b) {
                return d3.ascending(a.value, b.value);
            });
        globalBorough = avgByBorough;

        var xScale = d3.scaleBand()
            .domain(avgByBorough.map(d => d.key))
            .range([0, width])
            .padding(0.5);

        var yScale = d3.scaleLinear()
            .domain([0, d3.max(avgByBorough, d => d.value)])
            .range([height, 0]);

        var flipxScale = d3.scaleLinear()
            .domain([0, d3.max(avgByBorough, d => d.value)])
            .range([0, width]);

        var flipyScale = d3.scaleBand()
            .domain(avgByBorough.map(d => d.key))
            .range([0, height])
            .padding(0.5);

        var xAxis = d3.axisBottom()
            .scale(xScale);

        var yAxis = d3.axisLeft().ticks(7)
            .scale(yScale);

        var flipxAxis = d3.axisBottom().ticks(7)
            .scale(flipxScale);

        var flipyAxis = d3.axisLeft()
            .scale(flipyScale);
        var rectCanvas = gCanvas.append("g").attr("class", "rects");


        rectCanvas.selectAll("rect")
            .data(avgByBorough)
            .enter()
            .append("rect")
            .attr("class", d => d.key)
            .attr("x", d => xScale(d.key))
            .attr("y", d => yScale(d.value))
            .attr("width", xScale.bandwidth())
            .attr("height", d => height - yScale(d.value))
            .attr("style", "outline: thin solid black;")
            .on("mouseover", function() {
                tooltip.style("display", null);
            })
            .on("mouseout", function() {
                tooltip.style("display", "none");
            })
            .on("mousemove", function(d) {
                var xPosition = d3.mouse(this)[0] - 5;
                var yPosition = d3.mouse(this)[1] - 25;
                tooltip.attr("transform", "translate(" + xPosition + "," + yPosition + ")");
                tooltip.select("text").text("Average Price: $" + formatDecimals(d.value));
            })
            .attr("fill", d => colorScale(d.key))
            .on("click", function() {
                var borough = d3.select(this).attr("class");
                d3.csv(borough + ".csv", cvert, function(dataset2) {

                    var avgByNeighborhood = d3.nest()
                        .key(function(d) {
                            return d.neighbourhood;
                        })
                        .rollup(function(v) {
                            return d3.mean(v, function(d) {
                                return d.price;
                            });
                        })
                        .entries(dataset2).sort(function(a, b) {
                            return d3.ascending(a.value, b.value);
                        }).slice(0, 10);



                    xScale.domain(avgByNeighborhood.map(d => d.key))
                    yScale.domain([0, d3.max(avgByNeighborhood, d => d.value)])

                    flipxScale.domain([0, d3.max(avgByNeighborhood, d => d.value)])
                    flipyScale.domain(avgByNeighborhood.map(d => d.key))
                    if (!flipped) {
                        rectCanvas.selectAll("rect")
                            .data(avgByNeighborhood).enter()
                            .append("rect")
                            .attr("class", d => d.key)
                            .attr("x", d => xScale(d.key))
                            .attr("y", d => yScale(d.value))
                            .attr("width", xScale.bandwidth())
                            .attr("height", d => height - yScale(d.value))
                            .attr("fill", d => colorScale(borough))
                            .attr("style", "outline: thin solid black;")
                            .on("mouseover", function() {
                                tooltip.style("display", null);
                            })
                            .on("mouseout", function() {
                                tooltip.style("display", "none");
                            })
                            .on("mousemove", function(d) {
                                var xPosition = d3.mouse(this)[0] - 5;
                                var yPosition = d3.mouse(this)[1] - 25;
                                tooltip.attr("transform", "translate(" + xPosition + "," + yPosition + ")");
                                tooltip.select("text").text("Average Price: $" + formatDecimals(d.value));
                            })
                            .merge(gCanvas.selectAll("rect").data(avgByNeighborhood))
                            .transition()
                            .duration(1000)
                            .attr("class", d => d.key)
                            .attr("x", d => xScale(d.key))
                            .attr("y", d => yScale(d.value))
                            .attr("fill", d => colorScale(borough))
                            .attr("width", xScale.bandwidth())
                            .attr("height", d => height - yScale(d.value));


                        //Update X axis
                        gCanvas.select(".xaxis")
                            .transition()
                            .duration(1000)
                            .call(xAxis);

                        //Update Y axis
                        gCanvas.select(".yaxis")
                            .transition()
                            .duration(1000)
                            .call(yAxis);
                    } else {
                        rectCanvas.selectAll("rect")
                            .data(avgByNeighborhood).enter()
                            .append("rect")
                            .attr("class", d => d.key)
                            .attr("x", 0)
                            .attr("y", d => flipyScale(d.key))
                            .attr("width", d => flipxScale(d.value))
                            .attr("height", flipyScale.bandwidth())
                            .attr("fill", d => colorScale(borough))
                            .attr("style", "outline: thin solid black;")
                            .on("mouseover", function() {
                                tooltip.style("display", null);
                            })
                            .on("mouseout", function() {
                                tooltip.style("display", "none");
                            })
                            .on("mousemove", function(d) {
                                var xPosition = d3.mouse(this)[0] - 5;
                                var yPosition = d3.mouse(this)[1] - 25;
                                tooltip.attr("transform", "translate(" + xPosition + "," + yPosition + ")");
                                tooltip.select("text").text("Average Price: $" + formatDecimals(d.value));
                            })
                            .merge(gCanvas.selectAll("rect").data(avgByNeighborhood))
                            .transition()
                            .duration(1000)
                            .attr("class", d => d.key)
                            .attr("x", 0)
                            .attr("y", d => flipyScale(d.key))
                            .attr("fill", d => colorScale(borough))
                            .attr("width", d => flipxScale(d.value))
                            .attr("height", flipyScale.bandwidth());


                        //Update X axis
                        gCanvas.select(".xaxis")
                            .transition()
                            .duration(1000)
                            .call(flipxAxis);

                        //Update Y axis
                        gCanvas.select(".yaxis")
                            .transition()
                            .duration(1000)
                            .call(flipyAxis);
                    }




                })
            });

        d3.select("#flip")
            .on("click", function() {
                flipped = !flipped;
                if (flipped) {
                    rectCanvas.selectAll("rect").transition().duration(1000)
                        .attr("x", 0)
                        .attr("y", d => flipyScale(d.key))
                        .attr("width", d => flipxScale(d.value))
                        .attr("height", flipyScale.bandwidth());

                    gCanvas.select(".xaxis")
                        .transition()
                        .duration(1000)
                        .call(flipxAxis);

                    gCanvas.select(".yaxis")
                        .transition()
                        .duration(1000)
                        .call(flipyAxis);

                    gCanvas.select(".priceLabel").transition()
                        .attr("y", height + 15)
                        .attr("x", width / 2)
                        .attr("transform", "rotate(0)");

                } else {
                    rectCanvas.selectAll("rect").transition().duration(1000)
                        .attr("x", d => xScale(d.key))
                        .attr("y", d => yScale(d.value))
                        .attr("width", xScale.bandwidth())
                        .attr("height", d => height - yScale(d.value));

                    gCanvas.select(".xaxis")
                        .transition()
                        .duration(1000)
                        .call(xAxis);

                    gCanvas.select(".yaxis")
                        .transition()
                        .duration(1000)
                        .call(yAxis);

                    gCanvas.select(".priceLabel").transition()
                        .attr("transform", "rotate(-90)")
                        .attr("y", -45)
                        .attr("x", 0 - (height / 2));

                }

            });

        d3.select("#resetButton")
            .on("click", function() {
                xScale.domain(globalBorough.map(d => d.key))
                yScale.domain([0, d3.max(globalBorough, d => d.value)])
                flipxScale.domain([0, d3.max(globalBorough, d => d.value)]);
                flipyScale.domain(globalBorough.map(d => d.key));
                if (!flipped) {
                    gCanvas.transition()
                        .duration(1000).select(".xaxis").call(xAxis);
                    gCanvas.transition()
                        .duration(1000).select(".yaxis").call(yAxis);
                    rectCanvas.selectAll("rect")
                        .data(globalBorough).merge(gCanvas.selectAll("rect").data(globalBorough))
                        .transition()
                        .duration(1000)
                        .attr("class", d => d.key)
                        .attr("x", d => xScale(d.key))
                        .attr("y", d => yScale(d.value))
                        .attr("fill", d => colorScale(d.key))
                        .attr("width", xScale.bandwidth())
                        .attr("height", d => height - yScale(d.value));


                } else {
                    gCanvas.transition()
                        .duration(1000).select(".xaxis").call(flipxAxis);
                    gCanvas.transition()
                        .duration(1000).select(".yaxis").call(flipyAxis);
                    rectCanvas.selectAll("rect")
                        .data(globalBorough).merge(gCanvas.selectAll("rect").data(globalBorough))
                        .transition()
                        .duration(500)
                        .attr("class", d => d.key)
                        .attr("x", 0)
                        .attr("y", d => flipyScale(d.key))
                        .attr("fill", d => colorScale(d.key))
                        .attr("width", d => flipxScale(d.value))
                        .attr("height", flipyScale.bandwidth());


                }

                rectCanvas.selectAll("rect")
                    .data(globalBorough).exit()
                    .transition()
                    .duration(650)
                    .attr("x", width)
                    .remove();

            });


        gCanvas.append("g")
            .attr("class", "xaxis")
            .attr("transform", "translate(0," + (height) + ")")
            .style("font-size", "0.6em")
            .call(xAxis);

        gCanvas.append("g")
            .attr("class", "yaxis")
            .style("font-size", "0.6em")
            .call(yAxis);

        gCanvas.append("text")
            .attr("class", "priceLabel")
            .attr("transform", "rotate(-90)")
            .attr("y", -45)
            .attr("x", 0 - (height / 2))
            .attr("dy", "0.8em")
            .style("text-anchor", "middle")
            .text("Average Price(in US dollars)");

        var tooltip = gCanvas.append("g")
            .attr("class", "tooltipBar")
            .style("display", "none");

        tooltip.append("text")
            .attr("x", 30)
            .attr("dy", "1.2em")
            .style("text-anchor", "middle")
            .attr("font-size", "12px")
            .attr("font-weight", "bold");

        var legend = gCanvas.append("g")
            .attr("font-family", "sans-serif")
            .attr("font-size", 8)
            .attr("text-anchor", "end")
            .selectAll("g")
            .data(avgByBorough)
            .enter().append("g")
            .attr("transform", function(d, i) {
                return "translate(0," + i * 20 + ")";
            });

        legend.append("rect")
            .attr("x", width + 14)
            .attr("width", 20)
            .attr("height", 19)
            .attr("fill", d => colorScale(d.key));

        legend.append("text")
            .attr("x", width + 12)
            .attr("y", 9.5)
            .attr("dy", "0.12em")
            .text(function(d) {
                return d.key;
            });

    });
}
makeTheGraph()
});

<IPython.core.display.Javascript object>

Effectiveness for end user:

This plot is useful to the end user because it will allow them to browse price features at various depth levels. The user can see the average price of each borough, the exact amount on mouseover, and clicking on the bar will let them view the average price of the ten cheapest neighborhoods, letting them see the cheapest prices very clearly on macro and micro scale.The user can flip back and see different neighborhoods, allowing them more power over what they can browse. The color legends will serve to remind the user what borough they are currently exploring should they lose track. The flip function will allow the user to explore the data in the form that makes the most visual sense to them. The amount of filtering and navigation features on this graph are intended to greatly improve the users capability for browsing.

How:

Idiom - Bar Chart. Express value attribute with aligned vertical position, separate key attribute with horizontal position. Sort bars by value attribute.

Marks:

Lines - Bars (thick lines) are used to denote quantitative attribute

Channels:

Length - Length of the bars denote amount of quantitative data per each cateogrical attribute

Spatial Region - Used to express categories (boroughs/neighbourhood). Highest ranked channel in munzners effectiveness of categorical attributes

Color Hue - Used to denote the "meta" categories of each bar(borough)

Other Encodings:

Filter - Exact data available on mouseover

Embed - Button click on bars to transition into sub-category of the bar. Can also reset this.

Manipulate - Change - Button click to rearrange bars from vertical to horizontal

Effectiveness tied to HOW elements: Bars use length to denote amount of qualitative attribute for each categorical. Since the graph is following Munzner's Seperate, Ordered, and Aligned rule of thumb (for both the meta and embedded categories), the relative value for each category compared to each other is clear. That the inner categories are embedded ,and being able to go between the embedded data and original data easily, increases the data available in the visualization without causing information overload. Color hue referencing the meta category also ensures this information is available even when browsing the embedded data. Changing the data from vertical to horizontal and back does nothing to change how the data is represented, but it allows the user to modify the visualization to better suit their needs without clogging the graph with too many options.

# Conclusion and Critique

Critique:

Interactive map: I am mostly satisfied with this visualization. It provides much information to the user while allowing them to filter their view in various ways to avoid information overload. One improvement I would have liked to have was to make use of the **shape channel** to display the room type of each of the rental units, rather than having circles for all and having to mouseover the point to view room type. Right now, as this information is only accessible one unit at a time, the user can't draw conclusions about the spread of room types throughout the city. Was not done due to time constraints and complexity of task.

Box Plot: I believe this plot is effective at what it shows, but it is limited in that it shows only boroughs prices instead of individual neighborhoods. With more time I would have liked to add the feature of the interactive bar chart to "drill down" into the individual neighborhoods within each borough, allowing the user to browse more features of the data.

Stack Bar Chart: I believe that this graph would have been more effective if the different data had been **derived** from the original dataset before being graphed. While the plot largely functions to show relative distribution of room types for each borough, I believe the relative percentage for each borough would have been a more useful figure. The fact that Manhattan has many more places available for rent than Staten Island seems like an obvious assumption,or at least made obvious by the preceeding datasets. Meanwhile the number of shared rooms on Staten Island can't even be seen on this graph. A chart made using the relative values for each borough would have more plainly shown the distribution of room types for each borough. As well I believe that the different view, that moves all the stack types to a baseline with each other, is an ineffective use of the **spatial region** channel. While the effect of the bars moving and shrinking to line up vertically is a neat effect, the resulting graph doesn't has the bars spaced to far apart to easily tell the size difference between them, as well as a bar from the middle layer being in the same vertical space as the top layer.

Interactive Bar Chart: I believe this plot is an effective way for the user to examine the lowest average priced rental units across boroughs and neighborhoods, with the interactive elements being helpful to the user in filtering what data they want to see, and how they want to see it. I would have liked to show more variables than the two shown (price vs neigbourhood.borough), but the bar chart form doesn't lend itself to utilizing many channels without information overload for the user.

Overall:  The main critique with my graphs is that, besides the map, they show only two-three variables at once. While this makes them legible for the user, and allows them to browse the features of those variables effectively, most of the graphs do not show all the possible variables important to the users value judgment as mentioned in the premise.

Conclusion:

I believe that, as a whole, these 4 graphs would do a good job at giving the user in the premise an idea as to where they might try to find a room in New York, as between them all, the graphs show price distributions, as well as locations of interest and rough quality of units in the city. None of the graphs are unsuccessful in executing the Action-Target pair of Browse Features. The What and Why analyses drive the How of what is  placed in the graphs.
This exercise has shown that, while making a "standard" graph using the d3 library involves much more work than using a graphing framework like plotly,it has immense power for custom graphics and visualizations. I believe that if I was to start this assignment from scrath, I would do a much better job at addressing the critiques of my current graphs, as I learned much of the d3 library in the process of doing this assignment.